<div style="font-size: smaller; color: #808080; text-align: center; display: block;">MIT - 6.884 Computational Aspects of Therapeutics Design, Fall 2019</div>

# Problem Set 1: Junction Tree Variational Autoencoder for Molecular Graph Generation 
In this problem set, you will experiment with neural network learning parameters to train the variational auto-encoder presented in [Junction Tree Variational Autoencoder for Molecular Graph Generation](https://arxiv.org/pdf/1802.04364.pdf). Using this trained VAE model, you will generate a random sample of valid chemical structures and visualize the structures using rdkit. Before starting this problem set, please read the paper, being sure that you understand it well enought to have an understanding of the technique which is illustrated in this diagram:
![Figure 3](https://github.com/bh0085/6884-junctiontrees/blob/master/assets/ps1_fig3.png?raw=true)


# 0. INITIALIZATION

This code and all of the code which follows is designed to be run in a Google Cloud Compute Datalab environment, with- or without GPU support. Instructions for setup are located in the README document at the PS1 github repo, [here](https://github.com/bh0085/6884-junctiontrees). We have included tips and notes related to many of the common challenges initializing a Datalab environment. Please let us know if you have any questions! 

Once you have followed the Datalab initialization steps detailed in Github, you should be running this notebook inside of a functional Datalab environmetn. We will need to add some dependencies to get things working. To complete Part 0, simply run the following code to intialize your datalab environment for small molecule design with Rdkit, PyTorch 0.4.1 and all necessary dependencies and imports.

NOTE: **Want to run in your local environment?** This notebook should run in your local environment, however some dependencies of rdkit may require a package manager such as apt-get to install. If you're running in an ubuntu environment that should be fine. For users running in a mac environment, you will need to follow the rdkit instructions on [rdkit.org](https://www.rdkit.org) to get rdkit imports working. All users will need conda installed. The code in this notebook assumes a Datalab environment, but has notes where local users will probably have to make changes. 

NOTE: **Want to run on a GPU?** The ps1 README contains instructions for GPU & CPU setup. Running the GPU variant of this code may yield a 50% increase, but some of our students reported problems running `datalab beta create-gpu`. For those users, we have commented out the two places where the model uses CUDA. Let us know if you would like to debug GPU setup with us.

**Intialize datalab environment**  
Run this code block to execute bash commands initializing your datalab envoironment for rdkit and pytorch which will be required to run this set.

In [ ]:
!echo hi

In [ ]:
!chown root:root /tmp
!chmod 1777 /tmp
!echo "done fixing temp directory permissions"

In [ ]:
# Install system dependencies for rdkit to work
# @LOCAL-USERS: you may not need to run this code or to run these commands in the terminal.
# That's fine! Just make sure that your Rdkit imports work.
# You can test this by running: from rdkit import Chem.Draw
!apt-get update
!apt-get install libfontconfig1 libxrender1 --yes
!apt-get install libxext6 --yes
!echo "done installing dependencies"

In [ ]:
# Install conda dependencies in bash
# @LOCAL MACHINE USERS: Shell command support in Jupyter notebooks can be finicky. 
# You may find it more helpful to run these commands in the terminal.
!conda install pytorch=0.4.1 --yes
!conda install -c conda-forge rdkit --yes
!conda install -c rdkit nox --yes
!conda install -c rdkit cairo --yes
!echo "done installing conda packages"

**Initialize imports and system path variables**  
First, you will add to path the github repo which was downloaded below. 

NOTE: **@LOCAL MACHINE USERS**: This import is designed for the file structure of the google compute datalab environment, using an absolute path for the github repo cloned above. If you wish to run this code on your local machine, then you will need to change the path below to match the current working directory on your machine. 

In [ ]:
# Clone the PS1 dependencies
# if for some reason you need to re-run this code, rm -r the current github repo:
# rm -r 6884-junctiontrees
!git clone https://github.com/bh0085/6884-junctiontrees.git

In [ ]:
#Add PS1 dependencies to your python path
import sys, os
cwd = os.getcwd()
sys.path.append(os.path.join(cwd,'6884-junctiontrees'))
PS1_ROOT = os.path.join(cwd,"6884-junctiontrees")

NOTE: **PYTHON 2**: The imports in this problem set will only work in a python 2 environment. This will be the default environment for Datalab users. If you are running this code on your home machine, you will need to swith to a python 2 evnironment!

In [ ]:
#import torch, installed above
import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import DataLoader
from torch.autograd import Variable
import math, random, sys, os

#import jtnn from the github repo "./6884-junctiontrees", downloaded above
from jtnn import *

#import rdkit, installed above
import rdkit
import rdkit.Chem as Chem
from rdkit.Chem import Draw

lg = rdkit.RDLogger.logger() 
lg.setLevel(rdkit.RDLogger.CRITICAL)

# 1. PRETRAIN AUTOENCODER
In the [junction trees paper](https://arxiv.org/pdf/1802.04364.pdf), the zinc dataset is analyzed to create a library of chemically valid molecular subgraphs, termed atomic clusters. These clusters have been pre-computed for you in `vocab.txt` and were chosen to have sufficient diversity to be joined in trees to cover the vast majority of molecules in the Zinc or MOSES datasets, located at `/data`.  
<br/>

In this problem set we will be working with the zinc dataset (`./6884-junctiontrees/data/zinc`). Problem 1 loads data from the zinc dataset to pretrain a model. As described in the junction tree paper, the pretraining phase will involve learning an autoencoder neural network similar to the final Variational Auto-encoder, but lacking the regularizing beta parameter. This pre-trained model will be used in the next step to initialize a full variational auto-encoder model.  
<br/>

In [ ]:
# [PS1] @Students, the parameters below control the complexity of the neural network
# which is will be used to encode / decode junction trees in this project in addition 
# to representing the latent space--ie: the lossy representation of the molecular backbone--
# the choice of values will determine the speed of training and generalization (ie: test) 
# accuracy of the final outcome and influence the sampling results below. They will 
# also influence the speed of training significantly. In order to help you train larger sets,
# we have also included a truncated version of the "zinc" dataset (see NOTE in the code below).
# You can use this to experiment with larger hidden network sizes, but note that larger hidden
# networks may have different training requirements that smaller ones!
#
# ... Read the paper and use your past experience to choose parameter values that will work here.
#
# Hint: Experiment with some parameter values based upon the paper, and if you have a 
# hard time getting results, you can try the values posted in the github repository associated 
# with the paper, https://arxiv.org/pdf/1802.04364.
#

### YOUR CODE HERE ###
hidden_size= [...]
latent_size= [...]
depth=[...]
### END CUSTOM CODE ###

batch_size=40
stereo=1

DATA_PATH="../data/zinc"
OUT_DIR="../out/zinc"

# Calibration dataset paths
# @STUDENTS: Uncomment this code to run on a reduced set of
# training and testing data points. (see above)
#
# Training may take a long time to run.
# To help you experiment with the data and ensure that your network is running,
# you may want to test the code which follows on this limited dataset before
# running on the full "zinc" dataset. Each of these contains 1/25 of the
# molecules in training, testing, and validation sets.
#
# DATA_PATH="../data/zinc10k"
# OUT_DIR="../data/zinc10k"
#


if not os.path.isdir(OUT_DIR): os.makedirs(OUT_DIR)

VOCAB_PATH=os.path.join(DATA_PATH,"vocab.txt")
TRAIN_PATH=os.path.join(DATA_PATH,"train.txt")
TEST_PATH=os.path.join(DATA_PATH,"test.txt")

vocab = [x.strip("\r\n ") for x in open(VOCAB_PATH)] 
vocab = Vocab(vocab)


SAVE_PATH=os.path.join(OUT_DIR,"pretraining")
if not os.path.isdir(SAVE_PATH): os.makedirs(SAVE_PATH)
model = JTNNVAE(vocab, hidden_size, latent_size, depth)

for param in model.parameters():
    if param.dim() == 1:
        nn.init.constant(param, 0)
    else:
        nn.init.xavier_normal(param)

model = model#.cuda() #@GPU USERS: Uncomment .cuda() to run on this code on a GPU machine!
print "Model #Params: %dK" % (sum([x.nelement() for x in model.parameters()]) / 1000,)

optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = lr_scheduler.ExponentialLR(optimizer, 0.9)
scheduler.step()

dataset = MoleculeDataset(TRAIN_PATH)

MAX_EPOCH = 3
PRINT_ITER = 20

for epoch in xrange(MAX_EPOCH):
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4, collate_fn=lambda x:x, drop_last=True)

    word_acc,topo_acc,assm_acc,steo_acc = 0,0,0,0

    for it, batch in enumerate(dataloader):
        for mol_tree in batch:
            for node in mol_tree.nodes:
                if node.label not in node.cands:
                    node.cands.append(node.label)
                    node.cand_mols.append(node.label_mol)

        model.zero_grad()
        loss, kl_div, wacc, tacc, sacc, dacc = model(batch, beta=0)
        loss.backward()
        optimizer.step()

        word_acc += wacc
        topo_acc += tacc
        assm_acc += sacc
        steo_acc += dacc

        if (it + 1) % PRINT_ITER == 0:
            word_acc = word_acc / PRINT_ITER * 100
            topo_acc = topo_acc / PRINT_ITER * 100
            assm_acc = assm_acc / PRINT_ITER * 100
            steo_acc = steo_acc / PRINT_ITER * 100

            print "KL: %.1f, Word: %.2f, Topo: %.2f, Assm: %.2f, Steo: %.2f" % (kl_div, word_acc, topo_acc, assm_acc, steo_acc)
            word_acc,topo_acc,assm_acc,steo_acc = 0,0,0,0
            sys.stdout.flush()

    scheduler.step()
    print "learning rate: %.6f" % scheduler.get_lr()[0]
    torch.save(model.state_dict(), SAVE_PATH + "/model.iter-" + str(epoch))




# 2. Train Final VAE Model
Students, in this section you will train the Variational Autoencoder starting from the pre-initialized model in Section 1. Code in this section is mostly fleshed out. All you need to do is please fill in neural network parameters. In order to complete this section, please fill in the neural network parameters`beta` and `lr`. We recommend reading the paper to find suggested values for these quantities.

In [ ]:
# [PS1] 6.884 Students please experiment with values for regularization (beta) and learning rate (lr) based on the results of the paper. 
#
### YOUR CODE HERE ###

beta =[...]
lr =[...]

### YOUR CODE ENDS ###

VAE_SAVE_PATH=os.path.join(OUT_DIR,"vae_model")
MODEL_PATH=os.path.join(OUT_DIR,"pretraining/model.iter-2")
if not os.path.isdir(VAE_SAVE_PATH): os.makedirs(VAE_SAVE_PATH)
  
stereo = True 
model = JTNNVAE(vocab, hidden_size, latent_size, depth, stereo=stereo)

if MODEL_PATH is not None:
    model.load_state_dict(torch.load(MODEL_PATH))
else:
    for param in model.parameters():
        if param.dim() == 1:
            nn.init.constant(param, 0)
        else:
            nn.init.xavier_normal(param)

model = model#.cuda() #@GPU USERS: uncomment .cuda() to run on a GPU!
print "Model #Params: %dK" % (sum([x.nelement() for x in model.parameters()]) / 1000,)

optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = lr_scheduler.ExponentialLR(optimizer, 0.9)
scheduler.step()

dataset = MoleculeDataset(TRAIN_PATH)

MAX_EPOCH =7
PRINT_ITER = 20
for epoch in xrange(MAX_EPOCH):
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4, collate_fn=lambda x:x, drop_last=True)

    word_acc,topo_acc,assm_acc,steo_acc = 0,0,0,0

    for it, batch in enumerate(dataloader):
        for mol_tree in batch:
            for node in mol_tree.nodes:
                if node.label not in node.cands:
                    node.cands.append(node.label)
                    node.cand_mols.append(node.label_mol)

        try:
            model.zero_grad()
            loss, kl_div, wacc, tacc, sacc, dacc = model(batch, beta)
            loss.backward()
            optimizer.step()
        except Exception as e:
            print e
            continue

        word_acc += wacc
        topo_acc += tacc
        assm_acc += sacc
        steo_acc += dacc

        if (it + 1) % PRINT_ITER == 0:
            word_acc = word_acc / PRINT_ITER * 100
            topo_acc = topo_acc / PRINT_ITER * 100
            assm_acc = assm_acc / PRINT_ITER * 100
            steo_acc = steo_acc / PRINT_ITER * 100

            print "KL: %.1f, Word: %.2f, Topo: %.2f, Assm: %.2f, Steo: %.2f" % (kl_div, word_acc, topo_acc, assm_acc, steo_acc)
            word_acc,topo_acc,assm_acc,steo_acc = 0,0,0,0
            sys.stdout.flush()

        if (it + 1) % 15000 == 0: #Fast annealing
            scheduler.step()
            print "learning rate: %.6f" % scheduler.get_lr()[0]

        if (it + 1) % 1000 == 0: #Fast annealing
            torch.save(model.state_dict(), VAE_SAVE_PATH + "/model.iter-%d-%d" % (epoch, it + 1))

    scheduler.step()
    print "learning rate: %.6f" % scheduler.get_lr()[0]
    torch.save(model.state_dict(), VAE_SAVE_PATH + "/model.iter-" + str(epoch))




# 4. Sample new molecules
In the junction trees paper, the Variational Autoencode is used to sample small molecules from the computed distribution. These molecules are converted to "SMILES" strings and tested for validity. In this part of the code, you will sample that distribution, outputting molecules in SMILES format which will be tested for validity below.
<br/>

In [ ]:
#SAMPLE THE TRAINED VAE MODEL
# [PS1] STUDENTS @Problem set 1, simply choose a sample number to print SMILES strings!
### YOUR CODE HERE ###
#
nsample =100
#
### END CUSTOM CODE ###
VAE_MODEL_PATH = os.path.join(OUT_DIR,"vae_model/model.iter-4")
model = JTNNVAE(vocab, hidden_size, latent_size, depth, stereo=stereo)
load_dict = torch.load(VAE_MODEL_PATH)
missing = {k: v for k, v in model.state_dict().items() if k not in load_dict}
load_dict.update(missing) 
model.load_state_dict(load_dict)
model = model#.cuda()

torch.manual_seed(0)
samples = []
for i in xrange(nsample):
    s = model.sample_prior(prob_decode=False)
    samples.append(s)
    print(s)


# 5. Visualize the output molecules
This problem is more open-ended than previous parts. In the Junction Trees paper, the authors qualitatively evaluate the fidelity of molecular reconstructions by visualizing the small molecules which are sampled from the neighborhood of a chosen molecule. First to get a han As a consistency check on your results, utilize the Rdkit-based visualization tool from the Junction Trees paper to draw the randomly sampled molecules above.
<br/><br/>
Please fill in the code below to produce figures similar to the left side of Figure 6, below:

**Figure 6**
![Figure 6](https://github.com/bh0085/6884-junctiontrees/blob/master/assets/ps1_fig6.png?raw=true)

In [ ]:
### Your Code Here ###
# [PS1] @Students, please use rdkit or another suitable molecular visualization 
# to draw one or more of the molecules sampled above.
# 
# Hint: You may elect to use the methods of rdkit.Chem.Draw
#
##
### YOUR CODE HERE ###


### YOUR CODE ENDS ###


# Appendix: Helpful tips
<br/><br/>
**Cloud Datalab Docker Containers**
Using Cloud Datalab, you may find it helpful to access the source code & system files of the underlying docker container. In this problem it should not be necessary to touch these parts of the system, but if you are interested in doing so, a useful guide can be found [here](https://cloud.google.com/datalab/docs/how-to/working-with-notebooks#using_git_from_the_command_line)

<br/><br/>